In [58]:
import os
import sys
import cv2
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, Input
from keras.applications.vgg16 import VGG16
from force_utils import database, show_history

In [59]:
dataset = database('dataset/movie.mp4', 'dataset/forceinfo.csv')
force = dataset.make_tension()
frame_id, frames = dataset.make_inputs()
force_dic = {}
for i in range(len(frames)):
    force_dic[frame_id[i]] = force[i]
keys = list(force_dic.keys())
split_num = int(round(0.8 * len(keys)))
start_frame = keys[split_num]
max_frame = dataset.frame_count

In [60]:
input_shape = (299, 299, 3)
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))
x = base_model.output
x = Flatten(input_shape=base_model.output_shape[1:])(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(1)(x)

model = Model(inputs=base_model.input, outputs=prediction)
model.load_weights('dataset/weights/weights_02.h5')

In [61]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [66]:
class MovieProcess:

    # 動画保存の初期設定を行う
    def __init__(self, movie_file, target_dir, duration_second):
        self.movie_file = movie_file
        self.target_dir = target_dir
        self.video = cv2.VideoCapture(movie_file)
        self.fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
        self.frame_width = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        #self.frame_rate = self.video.get(cv2.CAP_PROP_FPS)
        self.frame_rate = 30
        self.duration = int(duration_second * self.frame_rate)

    # 目的のフレーム番号から指定した秒数だけ抜き出して保存する
    def extract(self, target_frame):
        self.video.set(1, target_frame)
        video_writer = cv2.VideoWriter(
            self.target_dir + self.movie_file.replace('.mp4', '').split('/')[-1] + '_' + str(target_frame) + '.mp4',
            self.fourcc,
            self.frame_rate,
            (self.frame_width, self.frame_height))
        for _ in range(self.duration):
            is_capturing, frame = self.video.read()
            frame_num = self.video.get(1)
            if is_capturing:
                
                img = Image.fromarray(np.uint8(frame[:,:,:])).resize((input_shape[0], input_shape[1]))
                img = np.asarray(img)
                img = img[np.newaxis, :, :, :]
                pred = model.predict(img / 255.0)[0][0]
                max_force = np.max(force)
                force_meter = pred / max_force

                if pred > 3:
                    color = (0,0,255)
                elif pred > 1.5:
                    color = (0,255,0)
                else:
                    color = (255,0,0)
                dst = cv2.rectangle(frame,
                                    (int(frame.shape[1]*0.85), int(frame.shape[0]*(-0.6*force_meter+0.8))),
                                    (int(frame.shape[1]*0.9), int(frame.shape[0]*0.8)),
                                    color, -1)
                dst = cv2.putText(dst, str(np.round(pred, 3)),
                                  (int(frame.shape[1]*0.85), int(frame.shape[0]*(-0.6*force_meter+0.78))),
                                  font, 1, color, 2, cv2.LINE_AA)
                dst = cv2.putText(dst, str(int(frame_num)),
                                  (int(frame.shape[1]*0.85), int(frame.shape[0]*0.85)),
                                  font, 1, (255,255,255), 2, cv2.LINE_AA)
                
                video_writer.write(dst)
                sys.stdout.write("\r%d%s%d" % (frame_num, ' / ', max_frame))
                sys.stdout.flush()
            else:
                print('the end of video')

In [71]:
ProcessSurgeryMovie = MovieProcess('./dataset/movie.mp4', './dataset/preds/', duration_second=100)

In [74]:
ProcessSurgeryMovie.extract(target_frame=18990)

21990 / 69821